In [7]:
import networkx as nx
import pandas as pd
# from dtl.graph_dtl import GraphDTL


class GraphBaseModule(object):
    def __init__(self, nodes=None, edges=None, read_from_db=False):
        self.nodes = nodes
        self.edges = edges

#         if read_from_db:
#             self.graph_dtl = GraphDTL()
#             self.nodes = self.graph_dtl.get_nodes()
#             self.edges = self.graph_dtl.get_edges


        if not (self.nodes and self.edges):
            self.nodes = pd.read_csv(
                "../sample_data/MajedAlasmariAP-nodes.csv")
            self.edges = pd.read_csv(
                "../sample_data/MajedAlasmariAP-edges.csv")

    def get_graph(self):
        return self.DG

    def create(self):
        self.DG = nx.DiGraph()

        for idx, node in self.nodes.iterrows():

            node_id = node['Id']
            del node['Id']

            attributes = {node_id: node}

            self.DG.add_node(node_id)
            nx.set_node_attributes(self.DG, attributes)

        for idx, edge in self.edges.iterrows():
            source, target = edge['Source'], edge['Target']
            del edge['Source'], edge['Target']

            attributes = {(source, target): edge}

            self.DG.add_edge(source, target)
            nx.set_edge_attributes(self.DG, attributes)

    def show_random(self):
        import matplotlib.pyplot as plt

        plt.figure(figsize=(15, 12))
        nx.draw_random(self.DG)
        plt.show()

    

In [8]:
# import networkx as nx
# from graph.base import GraphBaseModule


# compute on database
class MetricModule(GraphBaseModule):
    def __init__(self, DG=None):
        self.DG = DG
        if not self.DG:
            super(MetricModule, self).__init__()
            self.create()

    def eigen_centrality(self):
        eigen_centrality = nx.eigenvector_centrality(self.DG)
        return eigen_centrality

    def degree(self, node: str):
        outdeg = self.DG.out_degree(node)
        indeg = self.DG.in_degree(node)

        degree = {"outDegree": outdeg, "inDegree": indeg, "totalDegree": indeg+outdeg}
        return degree

    def weighted_degree(self, node: str, weight: str = "Weight"):
        outdeg = self.DG.out_degree(node, weight=weight)
        indeg = self.DG.in_degree(node, weight=weight)

        degree = {"outDegree": outdeg, "inDegree": indeg, "totalDegree": indeg+outdeg}
        return degree


In [9]:
import json


class QuerySerializer:
    def __init__(self, DG):
        self.DG = DG

    def serialize(self):
        query_result = dict()

        query_result["graph"] = self.create_graph_interface()
        query_result["user"] = self.create_user_data()

        return json.dumps(query_result)

    def create_graph_interface(self, node_new_feature=None, edge_new_feature=None):
        if node_new_feature:
            nodes = {node: features.update(node_new_feature) for node , features in self.DG.nodes(data=True)}
        else:
            nodes = {node: features for node , features in self.DG.nodes(data=True)}

        if edge_new_feature:
            edges = {source+"-->"+target: features.update({"from": source, "to":target}).update(edge_new_feature)
                    for source, target, features in self.DG.edges(data=True)}
        else:
            edges = {source+"-->"+target: features.update({"from": source, "to":target})
                        for source, target, features in self.DG.edges(data=True)}

        meta = self.create_graph_meta()
        return {"nodes": nodes,
                "edges": edges,
                "meta": meta}

    def create_graph_meta(self):
        settings = self.create_settings()
        filters = self.create_filters()
        centerNodeId = None

        return {"settings": settings,
                "filters": filters,
                "centerNodeId": centerNodeId}

    def create_settings(self, key: str=None, values=None):
        if not key:
            return None
        value = self.create_variable_properties(values)

        return {key: value}

    def create_filters(self, key: str=None, values=None):
        if not key:
            return None
        value = self.create_variable_properties()

        return {key: value}

    def create_variable_properties(self, values):
        return {
            "enable": values.get("enable", None),
            "editable": values.get("editable", None),
            "type": values.get("type", None),
            "options": values.get("options", None),
            "value": values.get("value", None),
            "children": values.get("children", None)
        }

    def create_user_data(self):
        return {"nodeCountLimit": None,
                "hasAccess": None}
